# ***Crawl thông tin url và title của bài báo***

In [47]:
import requests
from bs4 import BeautifulSoup
import csv
import time

headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
}

articles_data = []
all_seen_urls = set()  # Tập hợp global để theo dõi tất cả URL đã crawl

for page in range(1, 250):  # Crawl 4 trang đầu
    url = f'https://baochinhphu.vn/chinh-tri/trang-{page}.htm'
    print(f'Đang crawl {url}')
    response = requests.get(url, headers=headers)
    soup = BeautifulSoup(response.text, 'html.parser')

    page_articles = []

    # Crawl từ class="box-category-item"
    box_items = soup.find_all('div', class_='box-category-item')
    for item in box_items:
        a_tag = item.find('a')
        if a_tag and a_tag.get('href'):
            page_articles.append({
                'title': a_tag.get('title'),
                'url': 'https://baochinhphu.vn' + a_tag.get('href')
            })

    # Crawl từ class="list__main"
    list_main = soup.find('div', class_='list__main')
    if list_main:
        list_items = list_main.find_all('li')
        for li in list_items:
            a_tag = li.find('a')
            if a_tag and a_tag.get('href'):
                page_articles.append({
                    'title': a_tag.get('title'),
                    'url': 'https://baochinhphu.vn' + a_tag.get('href')
                })

    # Crawl từ class="box-stream-item"
    stream_items = soup.find_all('div', class_='box-stream-item')
    for item in stream_items:
        a_tag = item.find('a', class_='box-stream-link-with-avatar')
        if a_tag and a_tag.get('href'):
            page_articles.append({
                'title': a_tag.get('title'),
                'url': 'https://baochinhphu.vn' + a_tag.get('href')
            })

    # Nếu là trang 2 trở đi, bỏ qua 2 bài đầu tiên để tránh trùng lặp
    if page > 1:
        page_articles = page_articles[2:]
        print(f"Đã bỏ qua 2 bài đầu tiên của trang {page}")

    # Loại bỏ bài trùng lặp trong trang hiện tại và với các trang trước
    unique_articles = []
    for article in page_articles:
        if article['url'] not in all_seen_urls:
            all_seen_urls.add(article['url'])
            unique_articles.append(article)

    # Thêm vào danh sách dữ liệu
    for article in unique_articles:
        articles_data.append({
            'url': article['url'],
            'title': article['title']
        })
        print(f'✓ {article["title"]}')
        time.sleep(0.5)

    print(f"Trang {page}: Thu thập được {len(unique_articles)} bài mới\n")

# Lưu kết quả vào file CSV
csv_file = 'baochinhphu_chinhtri_simple.csv'
with open(csv_file, 'w', newline='', encoding='utf-8-sig') as f:
    writer = csv.DictWriter(f, fieldnames=['url', 'title'])
    writer.writeheader()
    writer.writerows(articles_data)

print(f'\n✔️ Đã lưu {len(articles_data)} bài báo vào {csv_file}')
print(f'Tổng số URL duy nhất: {len(all_seen_urls)}')

Đang crawl https://baochinhphu.vn/chinh-tri/trang-1.htm
✓ TRỰC TIẾP: Kỷ niệm trọng thể 100 năm ngày Báo chí cách mạng Việt Nam
✓ 100 năm báo chí cách mạng: Vì dân tộc, vì tương lai
✓ Báo chí Cách mạng Việt Nam: Một thế kỷ đồng hành cùng đất nước, vì đất nước
✓ Báo chí đối ngoại là một phần hữu cơ, không thể tách rời của công tác đối ngoại
✓ Hành động ngay, quyết liệt và hiệu quả để triển khai 25 dịch vụ công trực tuyến toàn trình
✓ Tổ chức lại phòng chuyên môn cấp xã, đơn vị sự nghiệp công lập của Cần Thơ, Hậu Giang và Sóc Trăng
✓ Ngành dầu khí phải khai thác, sử dụng hiệu quả tài nguyên quốc gia
✓ Phó Thủ tướng Lê Thành Long hoan nghênh các tập đoàn hàng đầu của Nga mở rộng sang các lĩnh vực hợp tác mới
✓ Kịp thời tháo gỡ khó khăn, vướng mắc do quy định của pháp luật
✓ Phiên chất vấn và trả lời chất vấn: Đúng và trúng vấn đề
✓ Báo chí cách mạng Việt Nam: Ngọn cờ tư tưởng, động lực to lớn góp phần đưa đất nước vững bước vào kỷ nguyên mới
✓ Lễ báo công dâng Bác nhân kỷ niệm 100 năm Ngày

# ***Crawl publish_time và summary từ url đã lấy***

## ***Demo lấy thử publish_time***

In [ ]:
import requests
from bs4 import BeautifulSoup

url = "https://baochinhphu.vn/thu-tuong-vung-bien-cua-asean-dang-phai-doi-mat-voi-nhung-thach-thuc-nghiem-trong-10225060923064568.htm"
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
}

response = requests.get(url, headers=headers)
soup = BeautifulSoup(response.text, 'html.parser')

# Tìm thẻ div chứa ngày giờ
date_div = soup.find('div', attrs={'data-role': 'publishdate'})
if date_div:
    # Lấy phần text trước và sau thẻ <span>
    contents = list(date_div.stripped_strings)
    if len(contents) >= 2:
        publish_date = contents[0]
        publish_time = contents[1]
        print(f"🗓️ Ngày giờ xuất bản: {publish_date} {publish_time}")
    else:
        print("⚠️ Không lấy được đầy đủ ngày và giờ.")
else:
    print("❌ Không tìm thấy ngày giờ xuất bản.")


🗓️ Ngày giờ xuất bản: 09/06/2025 23:09


## ***Demo lấy thử summary***

In [ ]:
import requests
from bs4 import BeautifulSoup

url = "https://baochinhphu.vn/thu-tuong-vung-bien-cua-asean-dang-phai-doi-mat-voi-nhung-thach-thuc-nghiem-trong-10225060923064568.htm"
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
}

response = requests.get(url, headers=headers)
soup = BeautifulSoup(response.text, 'html.parser')

# Tìm thẻ chứa summary
summary_tag = soup.find('h2', class_='detail-sapo')
if summary_tag:
    summary_text = summary_tag.get_text(strip=True)
    print("📝 Summary:", summary_text)
else:
    print("❌ Không tìm thấy summary.")


📝 Summary: (Chinhphu.vn) - Ngày 9/6, tại phiên toàn thể của Hội nghị cấp cao về đại dương của Liên hợp quốc lần thứ 3 (UNOC 3), được tổ chức tại thành phố Nice, Cộng hòa Pháp, Thủ tướng Chính phủ Phạm Minh Chính đã có bài phát biểu quan trọng đại diện cho 10 quốc gia Hiệp hội các quốc gia Đông Nam Á (ASEAN), đồng thời chia sẻ quan điểm của Việt Nam về bảo tồn và sử dụng bền vững biển, đại dương và tài nguyên biển.


# ***Giờ mới lấy thật nè con trai***

In [49]:
import requests
from bs4 import BeautifulSoup
import csv
import time

input_file = 'baochinhphu_chinhtri_simple.csv'
output_file = 'demo.csv'

headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
}

def extract_publish_date(soup):
    try:
        date_div = soup.find('div', attrs={'data-role': 'publishdate'})
        if date_div:
            contents = list(date_div.stripped_strings)
            if len(contents) >= 2:
                return f"{contents[0]} {contents[1]}"
        return ''
    except Exception as e:
        print(f"❌ Lỗi khi trích xuất publish_date: {e}")
        return ''

def extract_summary(soup):
    try:
        summary_tag = soup.find('h2', class_='detail-sapo')
        if summary_tag:
            summary_text = summary_tag.get_text(strip=True)
            # Loại bỏ tiền tố "(Chinhphu.vn) - " nếu có
            if summary_text.startswith("(Chinhphu.vn) - "):
                summary_text = summary_text[len("(Chinhphu.vn) - "):]
            return summary_text
        return ''
    except Exception as e:
        print(f"❌ Lỗi khi trích xuất summary: {e}")
        return ''

# Đọc dữ liệu từ file CSV đầu vào
articles = []
with open(input_file, 'r', encoding='utf-8-sig') as f:
    reader = csv.DictReader(f)
    for row in reader:
        articles.append(row)

# Cập nhật từng bài báo
for article in articles:
    url = article.get('url')
    if not url:
        continue
    try:
        print(f"🔎 Đang xử lý: {url}")
        res = requests.get(url, headers=headers, timeout=10)
        soup = BeautifulSoup(res.text, 'html.parser')

        publish_date = extract_publish_date(soup)
        summary = extract_summary(soup)

        article['publish_date'] = publish_date
        article['summary'] = summary

        print(f"✔️ {publish_date} | {summary[:60]}...")
        time.sleep(1)
    except Exception as e:
        print(f"⚠️ Lỗi khi xử lý {url}: {e}")
        article['publish_date'] = ''
        article['summary'] = ''

# Ghi ra file mới
with open(output_file, 'w', newline='', encoding='utf-8-sig') as f:
    fieldnames = list(articles[0].keys())
    writer = csv.DictWriter(f, fieldnames=fieldnames)
    writer.writeheader()
    writer.writerows(articles)

print(f"\n✅ Đã lưu kết quả vào: {output_file}")


🔎 Đang xử lý: https://baochinhphu.vn/truc-tiep-le-ky-niem-trong-the-100-nam-ngay-bao-chi-cach-mang-viet-nam-102250621084319019.htm
✔️ 21/06/2025 11:30 | Sáng nay 21/6, tại Trung tâm Hội nghị Quốc gia, diễn ra  Lễ ...
🔎 Đang xử lý: https://baochinhphu.vn/100-nam-bao-chi-cach-mang-vi-dan-toc-vi-tuong-lai-102250620140559604.htm
✔️ 21/06/2025 06:00 | Trong kỷ nguyên vươn mình của Dân tộc, nơi từng cá nhân, từn...
🔎 Đang xử lý: https://baochinhphu.vn/bao-chi-cach-mang-viet-nam-mot-the-ky-dong-hanh-cung-dat-nuoc-vi-dat-nuoc-102250609160743888.htm
✔️ 21/06/2025 08:06 | Nhân dịp kỷ niệm 100 năm Ngày Báo chí Cách mạng Việt Nam (21...
🔎 Đang xử lý: https://baochinhphu.vn/bao-chi-doi-ngoai-la-mot-phan-huu-co-khong-the-tach-roi-cua-cong-tac-doi-ngoai-102250620225641057.htm
✔️ 20/06/2025 22:59 | Nhân dịp kỷ niệm 100 năm Báo chí cách mạng Việt Nam, Thứ trư...
🔎 Đang xử lý: https://baochinhphu.vn/hanh-dong-ngay-quyet-liet-va-hieu-qua-de-trien-khai-25-dich-cong-truc-tuyen-toan-trinh-10225062018194847.

# ***Trích xuất thông tin từ phần summary***

## ***Những trường hợp có quy tắc***

In [14]:
import pandas as pd
import re

# Đọc file CSV
df = pd.read_csv("demo.csv")

def extract_event_info(summary):
    result = {
        "event_type": None,
        "trigger": None,
        "arg_subject": None,
        "arg_weapon": None,
        "arg_time": None,
        "arg_location": None
    }

    # Chuyển summary sang string và xử lý None/NaN
    summary = str(summary) if pd.notna(summary) else ""
    summary_lower = summary.lower()

    # Trigger
    triggers = {
      # Statement (Tuyên bố / Phát biểu)
      "phát biểu": "Statement",
      "tuyên bố": "Statement",
      "khẳng định": "Statement",
      "công bố": "Statement",
      "thông báo": "Statement",
      "nhấn mạnh": "Statement",
      "chỉ đạo": "Statement",
      "phát ngôn": "Statement",
      "cho biết": "Statement",
      "đưa ra ý kiến": "Statement",
      "thông tin": "Statement",
      "cam kết": "Statement",
      "bài viết": "Statement",

      # Legislation (Ban hành)
      "ban hành": "Legislation",
      "thông qua": "Legislation",
      "luật": "Legislation",
      "nghị quyết": "Legislation",
      "chính sách": "Legislation",
      "quy định": "Legislation",
      "nghị định": "Legislation",
      "quyết định": "Legislation",
      "đề xuất luật": "Legislation",
      "sửa đổi luật": "Legislation",
      "trình quốc hội": "Legislation",
      "dự thảo": "Legislation",

      # Election (Bầu cử)
      "bầu": "Election",
      "bổ nhiệm": "Election",
      "trúng cử": "Election",
      "được bầu": "Election",
      "đề cử": "Election",
      "giữ chức": "Election",
      "tái đắc cử": "Election",
      "đảm nhận": "Election",
      "trở thành": "Election",
      "nhậm chức": "Election",
      "bỏ phiếu": "Election",

      # Resign (Từ chức)
      "từ chức": "Resign",
      "thôi chức": "Resign",
      "xin nghỉ": "Resign",
      "rút lui": "Resign",
      "nghỉ hưu": "Resign",
      "bị miễn nhiệm": "Resign",
      "bị cách chức": "Resign",
      "bị đình chỉ": "Resign",
      "không tái tranh cử": "Resign",

      # Transfer of Power (Chuyển đổi quyền lực)
      "chuyển giao quyền lực": "Transfer Of Power",
      "nhậm chức": "Transfer Of Power",
      "kế nhiệm": "Transfer Of Power",
      "bàn giao": "Transfer Of Power",
      "tiếp quản": "Transfer Of Power",
      "lên nắm quyền": "Transfer Of Power",
      "nhận chức": "Transfer Of Power",
      "tiếp tục lãnh đạo": "Transfer Of Power",

      # Conflict / Attack (Tấn công)
      "tấn công": "Conflict/Attack",
      "đụng độ": "Conflict/Attack",
      "xung đột": "Conflict/Attack",
      "nổi dậy": "Conflict/Attack",
      "bạo loạn": "Conflict/Attack",
      "biểu tình bạo lực": "Conflict/Attack",
      "đe dọa": "Conflict/Attack",
      "sát hại": "Conflict/Attack",
      "bắt giữ": "Conflict/Attack",
      "ám sát": "Conflict/Attack",
      "cướp bóc": "Conflict/Attack",
      "phá hoại": "Conflict/Attack",
      "nổ súng": "Conflict/Attack",
      "xả súng": "Conflict/Attack",

      # Protest (Biểu tình)
      "biểu tình": "Protest",
      "xuống đường": "Protest",
      "phản đối": "Protest",
      "tuần hành": "Protest",
      "đình công": "Protest",
      "bãi công": "Protest",
      "chống đối": "Protest",
      "giương biểu ngữ": "Protest",
      "hô khẩu hiệu": "Protest",

      # Corruption (Tham nhũng)
      "tham nhũng": "Corruption",
      "hối lộ": "Corruption",
      "bê bối": "Corruption",
      "sai phạm": "Corruption",
      "gian lận": "Corruption",
      "rút ruột": "Corruption",
      "lạm dụng quyền lực": "Corruption",
      "bị điều tra": "Corruption",
      "bị khởi tố": "Corruption",
      "bị bắt": "Corruption",
      "bị truy tố": "Corruption",

      # Meeting (Gặp mặt)
      "gặp mặt": "Meeting",
      "hội đàm": "Meeting",
      "hội kiến": "Meeting",
      "họp": "Meeting",
      "thảo luận": "Meeting",
      "làm việc với": "Meeting",
      "trao đổi": "Meeting",
      "tiếp đón": "Meeting",
      "tiếp xúc": "Meeting",
      "tiếp kiến": "Meeting",
      "đàm phán": "Meeting",
      "gặp gỡ": "Meeting",

      # Cooperation (Hợp tác)
      "ký kết": "Cooperation",
      "hợp tác": "Cooperation",
      "thỏa thuận": "Cooperation",
      "ghi nhớ": "Cooperation",
      "ký biên bản": "Cooperation",
      "thống nhất": "Cooperation",
      "cùng phát triển": "Cooperation",
      "thúc đẩy hợp tác": "Cooperation",
      "cam kết": "Cooperation",

      # Visiting (Thăm thú)
      "công du": "Visiting",
      "chuyến thăm": "Visiting",
      "thăm chính thức": "Visiting",
      "viếng thăm": "Visiting",
      "sang thăm": "Visiting",
      "làm việc tại": "Visiting",
      "đón tiếp": "Visiting",
      "chào mừng": "Visiting",

      # Condolence (Lời chia buồn)
      "chia buồn": "Condolence",
      "gửi lời chia buồn": "Condolence",
      "tưởng niệm": "Condolence",
      "viếng": "Condolence",
      "phúng viếng": "Condolence",
      "truy điệu": "Condolence",
      "lễ tang": "Condolence",
      "mất mát": "Condolence",
      "đau buồn": "Condolence",

      # Award (Trao thưởng)
      "trao tặng": "Award",
      "tặng thưởng": "Award",
      "được nhận": "Award",
      "được vinh danh": "Award",
      "huân chương": "Award",
      "bằng khen": "Award",
      "danh hiệu": "Award",
      "ghi nhận": "Award",
      "tuyên dương": "Award"
}


    for trig_lower, evt_type in triggers.items():
        match = re.search(rf"\b{re.escape(trig_lower)}\b", summary_lower)
        if match:
            start_idx = match.start()
            end_idx = match.end()
            result["trigger"] = summary[start_idx:end_idx]
            result["event_type"] = evt_type
            break

    # Time
    time_patterns = [
        r"\b(Sáng nay|Chiều nay|Tối nay|Trưa nay|Sáng|Chiều|Tối|Trưa|sáng|chiều|tối|trưa)\s+\d{1,2}/\d{1,2}(?:/\d{2,4})?\b",  # Sáng nay 21/6 hoặc Sáng 21/6/2025
        r"\bNgày\s+\d{1,2}/\d{1,2}/\d{2,4}\b",    # Ngày 17/2/2021
        r"\bNgày\s+\d{1,2}/\d{1,2}\b",            # Ngày 17/2
        r"\b\d{1,2}/\d{1,2}/\d{2,4}\b",           # 21/6/1925
        r"\b\d{1,2}/\d{1,2}\b",                   # 21/6
        r"Từ\s+ngày\s+\d{1,2}\-\d{1,2}/\d{1,2}/\d{2,4}",   # Từ ngày 17-18/2/2022
        r"Từ\s+ngày\s+\d{1,2}\-\d{1,2}/\d{1,2}",           # Từ ngày 17-18/2
        r"Từ\s+\d{1,2}\-\d{1,2}/\d{1,2}/\d{2,4}",          # Từ 17-18/2/2022
        r"Từ\s+\d{1,2}\-\d{1,2}/\d{1,2}",                  # Từ 17-18/2
    ]

    for pattern in time_patterns:
        match = re.search(pattern, summary)
        if match:
            result["arg_time"] = match.group(0).strip()
            break


    # Location
    location_match = re.search(r"(tại\s[^,.;]*)", summary)
    if location_match:
        result["arg_location"] = location_match.group()

    # Subject - Thay đổi độ ưu tiên: Danh xưng + tên người trước, tên tổ chức sau
    # 1. Tìm danh xưng + tên người (ưu tiên cao nhất)
    long_titles_with_names = [
        # Lãnh đạo cấp cao
        "Tổng Bí thư",
        "Phó Thủ tướng Thường trực Chính phủ",
        "Phó Thủ tướng Chính phủ",
        "Thủ tướng Chính phủ",
        "Phó Chủ tịch nước",
        "Chủ tịch nước",
        "Phó Chủ tịch Quốc hội",
        "Chủ tịch Quốc hội",
        "Phó Thủ tướng",
        "Thủ tướng",
        "Phó Tổng Thư ký Quốc hội",
        "Bí thư Thành ủy",
        "Bí thư Tỉnh ủy",
        "Ủy viên Bộ Chính trị",
        "Ủy viên Trung ương Đảng",
        "Chánh án Tòa án Nhân dân Tối cao",
        "Viện trưởng Viện kiểm sát Nhân dân Tối cao",
        "Chủ nhiệm Văn phòng Chính phủ",
        "đại biểu Quốc hội",

        # Bộ, ngành
        "Bộ trưởng Bộ Quốc phòng",
        "Bộ trưởng Bộ Công an",
        "Bộ trưởng Bộ Ngoại giao",
        "Bộ trưởng Bộ Tài chính",
        "Bộ trưởng Bộ Giáo dục và Đào tạo",
        "Bộ trưởng Bộ Y tế",
        "Bộ trưởng Bộ Nội vụ",
        "Bộ trưởng Bộ Kế hoạch và Đầu tư",
        "Bộ trưởng Bộ Lao động - Thương binh và Xã hội",
        "Bộ trưởng",
        "Thứ trưởng Bộ Công Thương",
        "Thứ trưởng Bộ Thông tin và Truyền thông",
        "Thứ trưởng",

        # Địa phương
        "Phó Chủ tịch UBND",
        "Chủ tịch UBND",
        "Phó Chủ tịch HĐND",
        "Chủ tịch HĐND",

        # Quân đội, Công an
        "Đại tướng",
        "Thượng tướng",
        "Trung tướng",
        "Thiếu tướng",
        "Thượng tá",
        "Đại tá",

        # Quốc tế
        "Phó Tổng Thư ký Liên Hợp Quốc",
        "Tổng Thư ký Liên Hợp Quốc",
        "Đại sứ Hoa Kỳ tại Việt Nam",
        "Đại sứ",
        "Ngài",

        # Khác
        "Trưởng Ban Tuyên giáo và Dân vận Trung ương",
        "Trưởng Ban Tổ chức Trung ương",
        "Phó Giáo sư"
        "Giáo sư"
        "Tiến sĩ",
        "Thạc sĩ",
        "Nhà báo",
        "Nhà văn",
        "Ông",
        "Bà",
    ]

    for title in long_titles_with_names:
        pattern = rf"{title}(?:\s+[A-ZĐ][\w\-]+){{1,5}}"
        match = re.search(pattern, summary)
        if match:
            result["arg_subject"] = match.group(0).strip()
            break

    # 2. Tên tổ chức phổ biến (ưu tiên thấp hơn)
    if result["arg_subject"] is None:
        subject_orgs = [
            # Cơ quan Đảng
            "Ban Tổ chức Trung ương",
            "Ban Bí thư",
            "Ban Kinh tế Trung ương",
            "Ban Dân vận Trung ương",
            "Ban Nội chính Trung ương",
            "Ban Tuyên giáo Trung ương",
            "Ủy ban Kiểm tra Trung ương",
            "Ban Chỉ đạo Trung ương về phòng, chống tham nhũng, tiêu cực",
            
            # Cơ quan Chính phủ và Quốc hội
            "Văn phòng Chính phủ",
            "Ủy ban Thường vụ Quốc hội",
            "Ủy ban Đối ngoại của Quốc hội",
            "Hội đồng Dân tộc của Quốc hội",
            "Hội đồng Bầu cử Quốc gia",
            "Chính phủ",
            "Đại biểu HĐND",
            "Đại biểu Quốc hội",
            "ĐBQH",
            "HĐND",

            # Bộ, ngành
            "Bộ Ngoại giao",
            "Bộ Công an",
            "Bộ Quốc phòng",
            "Bộ Tài chính",
            "Bộ Giáo dục và Đào tạo",
            "Bộ Y tế",
            "Bộ Kế hoạch và Đầu tư",
            "Bộ Lao động - Thương binh và Xã hội",
            "Bộ Nông nghiệp và Phát triển nông thôn",
            "Bộ Tư pháp",
            "Bộ Giao thông vận tải",
            "Bộ Xây dựng",
            "Bộ Văn hóa, Thể thao và Du lịch",
            "Bộ Khoa học và Công nghệ",
            "Bộ Công Thương",
            "Bộ Tài nguyên và Môi trường",
            "Bộ Thông tin và Truyền thông",
            "Bộ Nội vụ",

            # Cơ quan trực thuộc, lực lượng vũ trang
            "Quân ủy Trung ương",
            "Công an nhân dân",
            "Quân đội nhân dân Việt Nam",
            "Bộ Tư lệnh Bộ đội Biên phòng",
            "Bộ Tổng Tham mưu",
            "Tổng cục Chính trị",
            "Cảnh sát cơ động",

            # Các tổ chức chính trị - xã hội
            "Mặt trận tổ quốc (MTTQ) và các tổ chức chính trị - xã hội"
            "Mặt trận Tổ quốc Việt Nam",
            "Mặt trận Tổ quốc",
            "Hội Cựu chiến binh Việt Nam",
            "Hội Liên hiệp Phụ nữ Việt Nam",
            "Trung ương Đoàn TNCS Hồ Chí Minh",
            "Tổng Liên đoàn Lao động Việt Nam",
            "Hội Liên hiệp Phụ nữ Việt Nam",
            "Hội Nông dân Việt Nam",
            "Hội Cựu chiến binh Việt Nam",
            "Liên hiệp các Hội Khoa học và Kỹ thuật Việt Nam",
            "Các tổ chức chính trị - xã hội",

            # Tổ chức khác
            "Cổng TTĐT Chính phủ",
            "Thông tấn xã Việt Nam",
            "Đài Tiếng nói Việt Nam",
            "Đài Truyền hình Việt Nam",
            "Học viện Chính trị quốc gia Hồ Chí Minh",
            "Viện Kiểm sát Nhân dân Tối cao",
            "Tòa án Nhân dân Tối cao",
        ]

        for org in subject_orgs:
            if org in summary:
                result["arg_subject"] = org
                break

    # 3. Nếu vẫn chưa có, thử match tổ chức như "Quốc hội khóa XV"
    if result["arg_subject"] is None:
        qh_match = re.search(r"(Quốc hội khóa\s+(?:[A-Z]+|\d+))", summary)
        if qh_match:
            result["arg_subject"] = qh_match.group(1).strip()

    # 4. Tổ chức có danh xưng dài (VD: "Phó Tổng Thư ký Liên Hợp Quốc", "Tổng Thư ký Liên Hợp Quốc", ...)
    if result["arg_subject"] is None:
        long_titles = [
            "Phó Tổng Thư ký Liên Hợp Quốc",
            "Tổng Thư ký Liên Hợp Quốc",
            "Chủ tịch nước",
            "Thủ tướng",
            "Phó Thủ tướng",
            "Bộ trưởng",
            "Đại sứ",
            "Chủ tịch UBND",
            "Tổng Bí thư",
            "Ngài"
        ]
        for title in long_titles:
            if title in summary:
                result["arg_subject"] = title
                break

    # 5. UBND tỉnh + tỉnh
    if result["arg_subject"] is None:
        ubnd_match = re.search(
            r"(UBND tỉnh\s+(?:[A-ZĐ][a-zàáạảãâấầậẩẫăắằặẳẵêếềệểễèéẹẻẽíìỉĩịóòỏõọôốồộổỗơớờợởỡúùủũụưứừựửữýỳỷỹỵ\-]+\s*)+)",
            summary
        )
        if ubnd_match:
            result["arg_subject"] = ubnd_match.group(1).strip()

    # Weapon
    for word in ["bom", "súng", "xe tải", "lựu đạn", "bom hạt nhân"]:
        match = re.search(rf"\b{word}\b", summary_lower)
        if match:
            result["arg_weapon"] = word
            break

    return result

# Áp dụng hàm với cả cột summary và publish_date
events = df.apply(lambda row: extract_event_info(row["summary"]), axis=1)
improved_df = pd.DataFrame(events.tolist())

# Gộp lại kết quả
final_df = pd.concat([df[["url", "title", "publish_date", "summary"]], improved_df], axis=1)

# Hiển thị kết quả
final_df

,url,title,publish_date,summary,event_type,trigger,arg_subject,arg_weapon,arg_time,arg_location
0,https://baochinhphu.vn/truc-tiep-le-ky-niem-tr...,TRỰC TIẾP: Kỷ niệm trọng thể 100 năm ngày Báo ...,21/06/2025 11:30,"Sáng nay 21/6, tại Trung tâm Hội nghị Quốc gia...",None,None,None,None,Sáng nay 21/6,tại Trung tâm Hội nghị Quốc gia
1,https://baochinhphu.vn/100-nam-bao-chi-cach-ma...,"100 năm báo chí cách mạng: Vì dân tộc, vì tươn...",21/06/2025 06:00,"Trong kỷ nguyên vươn mình của Dân tộc, nơi từn...",None,None,None,None,None,None
2,https://baochinhphu.vn/bao-chi-cach-mang-viet-...,Báo chí Cách mạng Việt Nam: Một thế kỷ đồng hà...,21/06/2025 08:06,Nhân dịp kỷ niệm 100 năm Ngày Báo chí Cách mạn...,None,None,Ban Tuyên giáo Trung ương,None,21/6/1925,None
3,https://baochinhphu.vn/bao-chi-doi-ngoai-la-mo...,"Báo chí đối ngoại là một phần hữu cơ, không th...",20/06/2025 22:59,Nhân dịp kỷ niệm 100 năm Báo chí cách mạng Việ...,Statement,nhấn mạnh,Thứ trưởng Bộ Ngoại,None,None,None
4,https://baochinhphu.vn/hanh-dong-ngay-quyet-li...,"Hành động ngay, quyết liệt và hiệu quả để triể...",20/06/2025 18:31,"Chiều 20/6, tại Trụ sở Chính phủ, Phó Thủ tướn...",Meeting,họp,Phó Thủ tướng Nguyễn Chí Dũng,None,Chiều 20/6,tại Trụ sở Chính phủ
...,...,...,...,...,...,...,...,...,...,...
5967,https://baochinhphu.vn/chu-tich-nuoc-du-chuong...,Chủ tịch nước dự Chương trình 'Xuân Biên phòng...,20/01/2024 23:23,"Tối 20/1, Chương trình “Xuân Biên phòng ấm lòn...",Statement,phát biểu,Chủ tịch nước Võ Văn Thưởng,None,Tối 20/1,tại TP
5968,https://baochinhphu.vn/thu-tuong-pham-minh-chi...,Thủ tướng Phạm Minh Chính tới thủ đô Bucharest...,20/01/2024 21:42,Sau khi kết thúc tốt đẹp chuyến thăm chính thứ...,Visiting,chuyến thăm,Thủ tướng Chính phủ Phạm Minh Chính,None,chiều 20/1,None
5969,https://baochinhphu.vn/thu-tuong-tham-trung-ta...,Thủ tướng thăm Trung tâm thương mại lớn nhất c...,20/01/2024 19:53,"Sáng 20/1, Thủ tướng Chính phủ Phạm Minh Chính...",None,None,Thủ tướng Chính phủ Phạm Minh Chính,None,Sáng 20/1,None
5970,https://baochinhphu.vn/viet-nam-la-mot-tam-guo...,Việt Nam là một tấm gương về con đường phát tr...,20/01/2024 19:02,"Sáng 20/1, tại Budapest, trong chương trình ch...",Visiting,chuyến thăm,Thủ tướng Phạm Minh Chính,None,Sáng 20/1,tại Budapest


In [13]:
df['summary'].iloc[5968]

'Sau khi kết thúc tốt đẹp chuyến thăm chính thức Hungary, chiều 20/1 (giờ địa phương), Thủ tướng Chính phủ Phạm Minh Chính và Phu nhân cùng đoàn đại biểu cấp cao Việt Nam đã tới Thủ đô Bucharest, bắt đầu thăm chính thức Romania theo lời mời của Thủ tướng Romania Ion-Marcel Ciolacu.'

In [15]:
text = 'Sáng nay 21/6, tại Trung tâm Hội nghị Quốc gia, diễn ra  Lễ kỷ niệm 100 năm ngày Báo chí cách mạng Việt Nam (21/6/1925 - 21/6/2025).'
extract_event_info(text)


{'event_type': None,
 'trigger': None,
 'arg_subject': None,
 'arg_weapon': None,
 'arg_time': 'Sáng nay 21/6',
 'arg_location': 'tại Trung tâm Hội nghị Quốc gia'}

## ***Những trường hợp không có quy tắc (Không thực hiện được)***

In [ ]:
final_df[final_df["event_type"].isna()]

In [ ]:
final_df[final_df['summary'].isna()]

In [ ]:
final_df[final_df['avg_subject'].isna()]

In [13]:
final_df['event_type'].unique()

array([None, 'Statement', 'Meeting', 'Cooperation', 'Legislation',
       'Condolence', 'Election', 'Visiting', 'Conflict/Attack',
       'Transfer Of Power', 'Award', 'Protest', 'Corruption', 'Resign'],
      dtype=object)

In [14]:
final_df['event_type'].value_counts()

event_type
Statement            1213
Meeting              1026
Legislation           881
Visiting              593
Cooperation           257
Election              217
Condolence            120
Award                  37
Transfer Of Power      28
Conflict/Attack        16
Corruption              4
Protest                 3
Resign                  3
Name: count, dtype: int64

In [ ]:
import pandas as pd
import numpy as np
import ast
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, classification_report
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import (
    XLMRobertaTokenizerFast,
    XLMRobertaForTokenClassification,
    TrainingArguments,
    Trainer,
    DataCollatorForTokenClassification
)
from torch.nn.utils.rnn import pad_sequence
import warnings
from collections import Counter, defaultdict
warnings.filterwarnings('ignore')

# Thiết lập device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

class ArgumentDetectionDataset(Dataset):
    def __init__(self, tokens_list, bio_tags_list, tokenizer, max_length=512, label_to_id=None):
        self.tokens_list = tokens_list
        self.bio_tags_list = bio_tags_list
        self.tokenizer = tokenizer
        self.max_length = max_length

        # Tạo label mapping (hoặc sử dụng mapping có sẵn)
        if label_to_id is None:
            all_tags = set()
            for bio_tags in bio_tags_list:
                all_tags.update(bio_tags)

            self.label_to_id = {tag: idx for idx, tag in enumerate(sorted(all_tags))}
        else:
            self.label_to_id = label_to_id

        self.id_to_label = {idx: tag for tag, idx in self.label_to_id.items()}
        self.num_labels = len(self.label_to_id)

        print(f"Found {self.num_labels} unique labels: {list(self.label_to_id.keys())}")

    def __len__(self):
        return len(self.tokens_list)

    def __getitem__(self, idx):
        tokens = self.tokens_list[idx]
        bio_tags = self.bio_tags_list[idx]

        # Tokenize và align labels
        encoding = self.tokenizer(
            tokens,
            is_split_into_words=True,
            padding='max_length',
            truncation=True,
            max_length=self.max_length,
            return_tensors='pt'
        )

        # Align labels với sub-word tokens
        labels = []
        word_ids = encoding.word_ids(batch_index=0)
        previous_word_idx = None

        for word_idx in word_ids:
            if word_idx is None:
                labels.append(-100)  # Special tokens
            elif word_idx != previous_word_idx:
                if word_idx < len(bio_tags):
                    labels.append(self.label_to_id[bio_tags[word_idx]])
                else:
                    labels.append(self.label_to_id['O'])
            else:
                labels.append(-100)  # Sub-word tokens
            previous_word_idx = word_idx

        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'labels': torch.tensor(labels, dtype=torch.long)
        }

def load_and_preprocess_data(file_path):
    """Load và preprocess dữ liệu từ CSV"""
    print("Loading data...")
    df = pd.read_csv(file_path)

    print(f"Dataset shape: {df.shape}")
    print(f"Columns: {df.columns.tolist()}")

    # Xử lý dữ liệu
    processed_data = []

    for idx, row in df.iterrows():
        try:
            # Parse tokens và bio_tags từ string representation
            if pd.isna(row['tokens']) or pd.isna(row['bio_tags']):
                continue

            tokens = ast.literal_eval(row['tokens']) if isinstance(row['tokens'], str) else row['tokens']
            bio_tags = ast.literal_eval(row['bio_tags']) if isinstance(row['bio_tags'], str) else row['bio_tags']

            # Kiểm tra độ dài khớp nhau
            if len(tokens) != len(bio_tags):
                print(f"Warning: Length mismatch at row {idx}: tokens={len(tokens)}, bio_tags={len(bio_tags)}")
                continue

            processed_data.append({
                'tokens': tokens,
                'bio_tags': bio_tags,
                'event_type': row.get('event_type', ''),
                'trigger': row.get('trigger', ''),
                'arg_subject': row.get('arg_subject', ''),
                'arg_location': row.get('arg_location', ''),
                'arg_time': row.get('arg_time', '')
            })

        except Exception as e:
            print(f"Error processing row {idx}: {e}")
            continue

    print(f"Successfully processed {len(processed_data)} samples")
    return processed_data

def create_stratification_key(bio_tags):
    """Tạo key cho stratified sampling dựa trên tần suất các labels"""
    # Đếm số lần xuất hiện của mỗi label
    label_counts = Counter(bio_tags)

    # Tạo key dựa trên các labels không phải 'O'
    non_o_labels = [label for label in label_counts.keys() if label != 'O']

    if not non_o_labels:
        return 'O_only'

    # Sắp xếp để tạo key nhất quán
    non_o_labels.sort()
    return '_'.join(non_o_labels)

def stratified_split_multilabel(data, test_size=0.2, dev_size=0.1, random_state=42):
    """Chia dữ liệu với stratified sampling cho multilabel"""
    print("Analyzing label distribution...")

    # Tạo stratification keys
    stratification_keys = []
    for item in data:
        key = create_stratification_key(item['bio_tags'])
        stratification_keys.append(key)

    # Phân tích distribution
    key_counts = Counter(stratification_keys)
    print(f"Found {len(key_counts)} unique label combinations:")
    for key, count in key_counts.most_common():
        print(f"  {key}: {count} samples")

    # Xử lý các keys có ít samples
    min_samples_per_split = 3  # Tối thiểu 3 samples cho mỗi split
    rare_keys = [key for key, count in key_counts.items() if count < min_samples_per_split]

    if rare_keys:
        print(f"Warning: {len(rare_keys)} label combinations have < {min_samples_per_split} samples")
        print("These will be handled specially to ensure all splits have representatives")

    # Chia dữ liệu
    indices = list(range(len(data)))

    try:
        # Thử stratified split bình thường trước
        train_indices, test_indices = train_test_split(
            indices,
            test_size=test_size,
            random_state=random_state,
            stratify=stratification_keys
        )

        # Chia train thành train và dev
        train_keys = [stratification_keys[i] for i in train_indices]
        train_indices, dev_indices = train_test_split(
            train_indices,
            test_size=dev_size/(1-test_size),
            random_state=random_state,
            stratify=train_keys
        )

    except ValueError as e:
        print(f"Stratified split failed: {e}")
        print("Falling back to manual stratified splitting...")

        # Manual stratified splitting
        train_indices, dev_indices, test_indices = manual_stratified_split(
            data, stratification_keys, test_size, dev_size, random_state
        )

    # Tạo datasets
    train_data = [data[i] for i in train_indices]
    dev_data = [data[i] for i in dev_indices]
    test_data = [data[i] for i in test_indices]

    print(f"Data split:")
    print(f"  Train: {len(train_data)} samples")
    print(f"  Dev: {len(dev_data)} samples")
    print(f"  Test: {len(test_data)} samples")

    # Kiểm tra label distribution
    check_label_distribution(train_data, dev_data, test_data)

    return train_data, dev_data, test_data

def manual_stratified_split(data, stratification_keys, test_size, dev_size, random_state):
    """Manual stratified splitting khi sklearn không hoạt động"""
    np.random.seed(random_state)

    # Nhóm indices theo keys
    key_to_indices = defaultdict(list)
    for idx, key in enumerate(stratification_keys):
        key_to_indices[key].append(idx)

    train_indices = []
    dev_indices = []
    test_indices = []

    for key, indices in key_to_indices.items():
        np.random.shuffle(indices)
        n_samples = len(indices)

        if n_samples == 1:
            # Chỉ có 1 sample, đưa vào train
            train_indices.extend(indices)
        elif n_samples == 2:
            # 2 samples: 1 train, 1 test
            train_indices.append(indices[0])
            test_indices.append(indices[1])
        else:
            # Chia theo tỷ lệ
            n_test = max(1, int(n_samples * test_size))
            n_dev = max(1, int(n_samples * dev_size))
            n_train = n_samples - n_test - n_dev

            # Đảm bảo ít nhất 1 sample cho mỗi split
            if n_train <= 0:
                n_train = 1
                n_test = max(1, (n_samples - n_train) // 2)
                n_dev = n_samples - n_train - n_test

            test_indices.extend(indices[:n_test])
            dev_indices.extend(indices[n_test:n_test + n_dev])
            train_indices.extend(indices[n_test + n_dev:])

    return train_indices, dev_indices, test_indices

def check_label_distribution(train_data, dev_data, test_data):
    """Kiểm tra phân phối labels trong các splits"""
    def get_all_labels(data):
        all_labels = set()
        for item in data:
            all_labels.update(item['bio_tags'])
        return all_labels

    train_labels = get_all_labels(train_data)
    dev_labels = get_all_labels(dev_data)
    test_labels = get_all_labels(test_data)

    all_labels = train_labels.union(dev_labels).union(test_labels)

    print(f"\nLabel distribution check:")
    print(f"  Total unique labels: {len(all_labels)}")
    print(f"  Train labels: {len(train_labels)}")
    print(f"  Dev labels: {len(dev_labels)}")
    print(f"  Test labels: {len(test_labels)}")

    missing_in_train = all_labels - train_labels
    missing_in_dev = all_labels - dev_labels
    missing_in_test = all_labels - test_labels

    if missing_in_train:
        print(f"  Labels missing in train: {missing_in_train}")
    if missing_in_dev:
        print(f"  Labels missing in dev: {missing_in_dev}")
    if missing_in_test:
        print(f"  Labels missing in test: {missing_in_test}")

    if not (missing_in_train or missing_in_dev or missing_in_test):
        print("  ✓ All labels present in all splits")
def compute_metrics(eval_pred, id_to_label):
    """Tính toán metrics"""
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=2)

    # Flatten và loại bỏ -100 labels
    true_predictions = []
    true_labels = []

    for prediction, label in zip(predictions, labels):
        for pred_id, label_id in zip(prediction, label):
            if label_id != -100:
                true_predictions.append(id_to_label[pred_id])
                true_labels.append(id_to_label[label_id])

    # Tính F1 score
    f1_micro = f1_score(true_labels, true_predictions, average='micro')
    f1_macro = f1_score(true_labels, true_predictions, average='macro')

    return {
        'f1_micro': f1_micro,
        'f1_macro': f1_macro,
        'accuracy': np.mean([p == l for p, l in zip(true_predictions, true_labels)])
    }

def train_model(train_dataset, dev_dataset, model_name='xlm-roberta-base'):
    """Training model"""
    print(f"Initializing model: {model_name}")

    tokenizer = XLMRobertaTokenizerFast.from_pretrained(model_name)
    model = XLMRobertaForTokenClassification.from_pretrained(
        model_name,
        num_labels=train_dataset.num_labels,
        id2label=train_dataset.id_to_label,
        label2id=train_dataset.label_to_id
    )

    # Data collator
    data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer)

    # Training arguments
    training_args = TrainingArguments(
        output_dir='pipeline_xlmr_argument_detection',
        num_train_epochs=3,
        per_device_train_batch_size=8,
        per_device_eval_batch_size=8,
        warmup_steps=500,
        weight_decay=0.01,
        logging_dir='pipeline_xlmr_argument_detection/logs',
        logging_steps=100,
        eval_strategy="epoch",
        save_strategy="epoch",
        load_best_model_at_end=True,
        metric_for_best_model="f1_macro",
        greater_is_better=True,
        save_total_limit=2,
        report_to=None  # Disable wandb
    )

    # Trainer
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=dev_dataset,
        tokenizer=tokenizer,
        data_collator=data_collator,
        compute_metrics=lambda eval_pred: compute_metrics(eval_pred, train_dataset.id_to_label)
    )

    print("Starting training...")
    trainer.train()

    return trainer, model, tokenizer

def evaluate_model(trainer, test_dataset):
    """Đánh giá model trên test set"""
    print("Evaluating on test set...")

    test_results = trainer.evaluate(test_dataset)
    print(f"Test Results:")
    for key, value in test_results.items():
        print(f"  {key}: {value:.4f}")

    # Detailed predictions
    predictions = trainer.predict(test_dataset)
    pred_labels = np.argmax(predictions.predictions, axis=2)

    # Convert to readable format
    true_predictions = []
    true_labels = []

    for i, (prediction, label) in enumerate(zip(pred_labels, predictions.label_ids)):
        pred_tags = []
        true_tags = []
        for pred_id, label_id in zip(prediction, label):
            if label_id != -100:
                pred_tags.append(test_dataset.id_to_label[pred_id])
                true_tags.append(test_dataset.id_to_label[label_id])

        true_predictions.append(pred_tags)
        true_labels.append(true_tags)

    # Classification report
    flat_true = [tag for tags in true_labels for tag in tags]
    flat_pred = [tag for tags in true_predictions for tag in tags]

    print("\nDetailed Classification Report:")
    print(classification_report(flat_true, flat_pred))

    return test_results, true_predictions, true_labels
def main():
    # Load data
    data = load_and_preprocess_data('results.csv')

    if len(data) == 0:
        print("No valid data found. Please check your CSV file.")
        return

    # Split data với stratified sampling
    train_data, dev_data, test_data = stratified_split_multilabel(data)

    # Create tokenizer
    tokenizer = XLMRobertaTokenizerFast.from_pretrained('xlm-roberta-base')

    # Tạo label mapping từ toàn bộ dữ liệu để đảm bảo consistency
    all_labels = set()
    for item in data:
        all_labels.update(item['bio_tags'])
    global_label_to_id = {tag: idx for idx, tag in enumerate(sorted(all_labels))}

    print(f"Global label mapping: {global_label_to_id}")

    # Create datasets với global label mapping
    train_tokens = [item['tokens'] for item in train_data]
    train_bio_tags = [item['bio_tags'] for item in train_data]
    train_dataset = ArgumentDetectionDataset(train_tokens, train_bio_tags, tokenizer, label_to_id=global_label_to_id)

    dev_tokens = [item['tokens'] for item in dev_data]
    dev_bio_tags = [item['bio_tags'] for item in dev_data]
    dev_dataset = ArgumentDetectionDataset(dev_tokens, dev_bio_tags, tokenizer, label_to_id=global_label_to_id)

    test_tokens = [item['tokens'] for item in test_data]
    test_bio_tags = [item['bio_tags'] for item in test_data]
    test_dataset = ArgumentDetectionDataset(test_tokens, test_bio_tags, tokenizer, label_to_id=global_label_to_id)

    # Train model
    trainer, model, tokenizer = train_model(train_dataset, dev_dataset)

    # Evaluate
    test_results, predictions, true_labels = evaluate_model(trainer, test_dataset)

    # Print final F1 scores
    print(f"\n=== FINAL RESULTS ===")
    print(f"Test F1 (Micro): {test_results['eval_f1_micro']:.4f}")
    print(f"Test F1 (Macro): {test_results['eval_f1_macro']:.4f}")
    print(f"Test Accuracy: {test_results['eval_accuracy']:.4f}")

    # Save model
    model.save_pretrained('pipeline_xlmr_argument_detection/final_model')
    tokenizer.save_pretrained('pipeline_xlmr_argument_detection/final_model')
    print("Model saved to 'pipeline_xlmr_argument_detection/final_model'")

    return trainer, model, tokenizer, test_results

if __name__ == "__main__":
    # Chạy pipeline
    trainer, model, tokenizer, results = main()

KeyboardInterrupt: 

In [ ]:
final_df.isna().sum()

# ***Đánh label cho event_type***

In [16]:
import pandas as pd

# Danh sách theo thứ tự để đánh nhãn one-hot
event_types = [
    'Statement', 'Meeting', 'Cooperation', 'Legislation', 'Condolence',
    'Election', 'Visiting', 'Conflict/Attack', 'Transfer Of Power',
    'Award', 'Protest', 'Corruption', 'Resign'
]

# Hàm chuyển event_type thành one-hot vector
def encode_event_type(event):
    label = [0] * len(event_types)
    if event in event_types:
        index = event_types.index(event)
        label[index] = 1
    return label

# Áp dụng vào DataFrame
final_df['label'] = final_df['event_type'].apply(encode_event_type)


In [17]:
final_df

,url,title,publish_date,summary,event_type,trigger,arg_subject,arg_weapon,arg_time,arg_location,label
0,https://baochinhphu.vn/truc-tiep-le-ky-niem-tr...,TRỰC TIẾP: Kỷ niệm trọng thể 100 năm ngày Báo ...,21/06/2025 11:30,"Sáng nay 21/6, tại Trung tâm Hội nghị Quốc gia...",None,None,None,None,Sáng nay 21/6,tại Trung tâm Hội nghị Quốc gia,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
1,https://baochinhphu.vn/100-nam-bao-chi-cach-ma...,"100 năm báo chí cách mạng: Vì dân tộc, vì tươn...",21/06/2025 06:00,"Trong kỷ nguyên vươn mình của Dân tộc, nơi từn...",None,None,None,None,None,None,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
2,https://baochinhphu.vn/bao-chi-cach-mang-viet-...,Báo chí Cách mạng Việt Nam: Một thế kỷ đồng hà...,21/06/2025 08:06,Nhân dịp kỷ niệm 100 năm Ngày Báo chí Cách mạn...,None,None,Ban Tuyên giáo Trung ương,None,21/6/1925,None,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
3,https://baochinhphu.vn/bao-chi-doi-ngoai-la-mo...,"Báo chí đối ngoại là một phần hữu cơ, không th...",20/06/2025 22:59,Nhân dịp kỷ niệm 100 năm Báo chí cách mạng Việ...,Statement,nhấn mạnh,Thứ trưởng Bộ Ngoại,None,None,None,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
4,https://baochinhphu.vn/hanh-dong-ngay-quyet-li...,"Hành động ngay, quyết liệt và hiệu quả để triể...",20/06/2025 18:31,"Chiều 20/6, tại Trụ sở Chính phủ, Phó Thủ tướn...",Meeting,họp,Phó Thủ tướng Nguyễn Chí Dũng,None,Chiều 20/6,tại Trụ sở Chính phủ,"[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
...,...,...,...,...,...,...,...,...,...,...,...
5967,https://baochinhphu.vn/chu-tich-nuoc-du-chuong...,Chủ tịch nước dự Chương trình 'Xuân Biên phòng...,20/01/2024 23:23,"Tối 20/1, Chương trình “Xuân Biên phòng ấm lòn...",Statement,phát biểu,Chủ tịch nước Võ Văn Thưởng,None,Tối 20/1,tại TP,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
5968,https://baochinhphu.vn/thu-tuong-pham-minh-chi...,Thủ tướng Phạm Minh Chính tới thủ đô Bucharest...,20/01/2024 21:42,Sau khi kết thúc tốt đẹp chuyến thăm chính thứ...,Visiting,chuyến thăm,Thủ tướng Chính phủ Phạm Minh Chính,None,chiều 20/1,None,"[0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0]"
5969,https://baochinhphu.vn/thu-tuong-tham-trung-ta...,Thủ tướng thăm Trung tâm thương mại lớn nhất c...,20/01/2024 19:53,"Sáng 20/1, Thủ tướng Chính phủ Phạm Minh Chính...",None,None,Thủ tướng Chính phủ Phạm Minh Chính,None,Sáng 20/1,None,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
5970,https://baochinhphu.vn/viet-nam-la-mot-tam-guo...,Việt Nam là một tấm gương về con đường phát tr...,20/01/2024 19:02,"Sáng 20/1, tại Budapest, trong chương trình ch...",Visiting,chuyến thăm,Thủ tướng Phạm Minh Chính,None,Sáng 20/1,tại Budapest,"[0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0]"


# ***Lưu kết quả***

In [19]:
final_df['summary'] = final_df['summary'].str.replace(r'^\(Chinhphu\.vn\)\s*[-–]\s*', '', regex=True)
final_df.to_excel("final_output.xlsx", index=False)
final_df.to_csv("final_output.csv")

# ***Demo thử về dataset BKEE sẽ sử dụng trong bài toán***

In [41]:
import json

# Đọc file và chỉ lấy dòng đầu tiên
with open("BKEE-main/processed/dev.json", "r", encoding="utf-8") as f:
    first_item = json.loads(f.readline())

# Thông tin cơ bản
print(f"📄 doc_id   : {first_item.get('doc_id')}")
print(f"🆔 sent_id  : {first_item.get('sent_id')}")
print(f"📝 Sentence : {first_item.get('sentence')}")
print(f"🔡 Tokens   : {first_item.get('tokens')}")
print(f"📏 Token lens: {first_item.get('token_lens')}")
print()

# Thực thể (Entities)
if first_item.get("entity_mentions"):
    print("🔍 Entities:")
    for ent in first_item["entity_mentions"]:
        print(f"  - ID: {ent['id']}")
        print(f"    Text: {ent['text']}")
        print(f"    Type: {ent['entity_type']}")
        print(f"    Token span: [{ent['start']}, {ent['end']})")
        print(f"    Char span : [{ent['start_char']}, {ent['end_char']})")
else:
    print("🔍 Entities: None")
print()

# Sự kiện (Events)
if first_item.get("event_mentions"):
    print("⚡ Events:")
    for ev in first_item["event_mentions"]:
        print(f"  - ID: {ev['id']}")
        print(f"    Type: {ev['event_type']}")
        print(f"    Trigger: \"{ev['trigger']['text']}\" (tokens {ev['trigger']['start']}–{ev['trigger']['end']}, chars {ev['trigger']['start_char']}–{ev['trigger']['end_char']})")
        if ev.get("arguments"):
            print("    Arguments:")
            for arg in ev["arguments"]:
                print(f"      • {arg['role']}: \"{arg['text']}\" (Entity ID: {arg['entity_id']})")
        else:
            print("    Arguments: None")
else:
    print("⚡ Events: None")
print()

# Quan hệ (Relations)
if first_item.get("relation_mentions"):
    print("🔗 Relations:")
    for rel in first_item["relation_mentions"]:
        print(f"  - Relation between {rel['arg1_id']} and {rel['arg2_id']} (Type: {rel['relation_type']})")
else:
    print("🔗 Relations: None")

print("\n" + "=" * 100)


📄 doc_id   : dev-00000
🆔 sent_id  : dev-00000
📝 Sentence : Ông chủ Nhà Trắng hồi tháng 8/2017 ân xá cho Joe Arpaio , cảnh sát trưởng hạt Maricopa , bang Arizona , bị buộc tội không tuân lệnh toà án trong một vụ án hình sự liên quan đến những người tình nghi là dân nhập cư bất hợp pháp .
🔡 Tokens   : ['Ông', 'chủ', 'Nhà Trắng', 'hồi', 'tháng', '8/2017', 'ân xá', 'cho', 'Joe Arpaio', ',', 'cảnh sát', 'trưởng', 'hạt', 'Maricopa', ',', 'bang', 'Arizona', ',', 'bị', 'buộc tội', 'không', 'tuân', 'lệnh', 'toà án', 'trong', 'một', 'vụ', 'án', 'hình sự', 'liên quan', 'đến', 'những', 'người tình', 'nghi', 'là', 'dân', 'nhập cư', 'bất hợp pháp', '.']
📏 Token lens: [1, 1, 4, 1, 1, 2, 3, 1, 4, 1, 2, 1, 1, 3, 1, 1, 1, 1, 1, 2, 1, 2, 1, 3, 1, 1, 1, 1, 2, 2, 1, 1, 2, 1, 1, 1, 2, 3, 1]

🔍 Entities:
  - ID: dev-00000-T15
    Text: tháng 8/2017
    Type: TIME
    Token span: [4, 6)
    Char span : [22, 34)
  - ID: dev-00000-T14
    Text: Ông chủ Nhà Trắng
    Type: PER
    Token span: [0, 3)
    Char spa

{"doc_id": "test-00001", "sent_id": "test-00001", "tokens": ["Tôi", "là", "sinh viên", "vừa", "ra", "trường", ",", "tốt nghiệp", "đại học", "chuyên ngành", "cơ khí", ",", "đi", "làm", "đã", "hơn", "8", "tháng", "với", "mức", "lương", "7", "triệu", "đồng", "một", "tháng", "."], "sentence": "Tôi là sinh viên vừa ra trường , tốt nghiệp đại học chuyên ngành cơ khí , đi làm đã hơn 8 tháng với mức lương 7 triệu đồng một tháng .", "pieces": ["▁Tôi", "▁là", "▁sinh", "▁viên", "▁vừa", "▁ra", "▁trường", ",", "▁tốt", "▁nghiệp", "▁đại", "▁học", "▁chuyên", "▁ngành", "▁cơ", "▁khí", ",", "▁đi", "▁làm", "▁đã", "▁hơn", "▁8", "▁tháng", "▁với", "▁mức", "▁lương", "▁7", "▁triệu", "▁đồng", "▁một", "▁tháng", "."], "token_lens": [1, 1, 2, 1, 1, 1, 1, 2, 2, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], "entity_mentions": [], "event_mentions": [], "relation_mentions": []}


📄 doc_id   : dev-00000
🆔 sent_id  : dev-00000
📝 Sentence : Ông chủ Nhà Trắng hồi tháng 8/2017 ân xá cho Joe Arpaio , cảnh sát trưởng hạt Maricopa , bang Arizona , bị buộc tội không tuân lệnh toà án trong một vụ án hình sự liên quan đến những người tình nghi là dân nhập cư bất hợp pháp .
🔡 Tokens   : ['Ông', 'chủ', 'Nhà Trắng', 'hồi', 'tháng', '8/2017', 'ân xá', 'cho', 'Joe Arpaio', ',', 'cảnh sát', 'trưởng', 'hạt', 'Maricopa', ',', 'bang', 'Arizona', ',', 'bị', 'buộc tội', 'không', 'tuân', 'lệnh', 'toà án', 'trong', 'một', 'vụ', 'án', 'hình sự', 'liên quan', 'đến', 'những', 'người tình', 'nghi', 'là', 'dân', 'nhập cư', 'bất hợp pháp', '.']
📏 Token lens: [1, 1, 4, 1, 1, 2, 3, 1, 4, 1, 2, 1, 1, 3, 1, 1, 1, 1, 1, 2, 1, 2, 1, 3, 1, 1, 1, 1, 2, 2, 1, 1, 2, 1, 1, 1, 2, 3, 1]

🔍 Entities:
  - ID: dev-00000-T15
    Text: tháng 8/2017
    Type: TIME
    Token span: [4, 6)
    Char span : [22, 34)
  - ID: dev-00000-T14
    Text: Ông chủ Nhà Trắng
    Type: PER
    Token span: [0, 3)
    Char span : [0, 17)
  - ID: dev-00000-T16
    Text: Joe Arpaio , cảnh sát trưởng hạt Maricopa , bang Arizona
    Type: PER
    Token span: [8, 17)
    Char span : [45, 101)
  - ID: dev-00000-T17
    Text: không tuân lệnh toà án trong một vụ án hình sự liên quan đến những người tình nghi là dân nhập cư bất hợp pháp
    Type: CRIME
    Token span: [20, 38)
    Char span : [116, 226)

⚡ Events:
  - ID: dev-00000-T13
    Type: Pardon
    Trigger: "ân xá" (tokens 6–7, chars 35–40)
    Arguments:
      • Time: "tháng 8/2017" (Entity ID: dev-00000-T15)
      • Adjudicator: "Ông chủ Nhà Trắng" (Entity ID: dev-00000-T14)
      • Defendant: "Joe Arpaio , cảnh sát trưởng hạt Maricopa , bang Arizona" (Entity ID: dev-00000-T16)
      • Crime: "không tuân lệnh toà án trong một vụ án hình sự liên quan đến những người tình nghi là dân nhập cư bất hợp pháp" (Entity ID: dev-00000-T17)

🔗 Relations: None

====================================================================================================